In [43]:
from document_extractor import FinancialDocumentProcessor
from typing import TypedDict, Optional, Dict, Any

from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_openai import OpenAIEmbeddings, OpenAI
from langchain.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains import create_retrieval_chain


In [44]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [70]:
class ResultEvaluator():
    def __init__(self, 
        model_name: str = "gpt-4o-mini",
        openai_api_key: Optional[str] = None,
        temperature: float = 0.0,
        max_tokens: int = 512):
        
        self.model_name = model_name
        self.api_key = openai_api_key or os.getenv("OPENAI_API_KEY")
        self.temperature = temperature
        self.max_tokens = max_tokens

        self.llm = OpenAI(
            api_key=self.api_key,
            model=self.model_name,
            temperature=self.temperature,
            max_tokens=self.max_tokens,
        )

    def evaluate_bs_item(self, bs_item):
        
        try:
            assert bs_item["total_assets"] >= bs_item["total_liabilities"], "Liabilities exceed assets"
            assert bs_item["investment_properties"] <= bs_item["total_assets"], "Investment properties exceed total assets"
            assert bs_item["total_debt"] <= bs_item["total_liabilities"], "Debt exceeds liabilities"
            assert bs_item["nta_per_unit"] > 0, "NTA per unit is non-positive"
            return {"verdict": "Correct", "justification": "All checks passed"}
 

        except AssertionError as e:
            return {"verdict": "Incorrect", "justification": str(e)}

    
    def evaluate_is_item(self, is_item):
        
        try:
            assert is_item["total_revenue"] >= 0, "Total assets cannot be negative"
            return {"verdict": "Correct", "justification": "All checks passed"}

        except AssertionError as e:
            return {"verdict": "Incorrect", "justification": str(e)}

    
    # def evaluate(self,bs_item, is_item):
    #     system_prompt = """
    #     You are a rigorous auditor evaluating the output of a financial information extraction system. Your role is to assess whether the extracted financial data appears accurate, complete, and internally consistent, based solely on the provided extracted values.
    #     Return your verdict in the following format:
    #     ```json
    #     {
    #     "verdict": "Correct" or "Incorrect",
    #     "justification": "Short explanation of why the output was deemed correct or not."
    #     }

    #     """
        
    #     #  Follow these rules:
    #     # - Ensure no critical fields are missing or null.
    #     # - Flag obviously invalid or inconsistent values (e.g., negative assets, liabilities exceeding assets, missing totals).
    #     # - Check that units are plausible and the values are reasonable (e.g., NTA per unit should not be unreasonably large or small).
    #     # - Do not guess or fill in missing data.
    #     # - Respond with one of the following verdicts:
    #     #     - **"Correct"**: if the extracted data appears complete and consistent.
    #     #     - **"Incorrect"**: if there are inconsistencies, or values that cannot be trusted.
    #     print(bs_item)    

    #     prompt = f"{system_prompt} . The BS item is {str(bs_item)} and the IS item is {str(is_item)}. Evaluate the extracted information and provide your verdict."
    #     result = self.llm.invoke(prompt)
    #     print(result)
        

   

In [69]:
re = ResultEvaluator()
re.evaluate_bs_item(final_state["bs_result"])
re.evaluate_is_item(final_state["is_result"])

{'verdict': 'Correct', 'justification': 'All checks passed'}

In [71]:
class ExtractorState(TypedDict):
    file_name: str
    bs_result: Optional[dict]
    is_result: Optional[dict]
    feedback: Optional[str]
    retries: int

In [ ]:
from langchain_core.runnables import RunnableLambda

fdp = FinancialDocumentProcessor(data_dir="datasets/co_presentations")
evaluator = ResultEvaluator()

# Actor Node
def actor_node(state: ExtractorState) -> ExtractorState:
    print(f"Running actor on {state['file_name']}")

    fdp.ingest_document(state["file_name"])
    fdp.extract_tables()
    extracted_info = fdp.extract_information()
    # self.store_bs_item(extracted_info[0])
    # self.store_is_item(extracted_info[1])

    # extracted_info = fdp.ingest_extract_save(state["file_name"])

    return {
        **state,
        "bs_result": extracted_info[0],
        "is_result": extracted_info[1],
        "retries": state["retries"] + 1,
    }

# Critic Node
def critic_node(state: ExtractorState) -> ExtractorState:
    print("Critic evaluating...")
    
    verdict = evaluator.evaluate_bs_item(state["bs_result"])["verdict"] and evaluator.evaluate_is_item(state["is_result"])["verdict"]
    if verdict == "Correct":
        print("Critic: All checks passed.")
    else:
        print("Critic: Some checks failed.")
    return {**state, "feedback": verdict}



actor_runnable = RunnableLambda(actor_node)
critic_runnable = RunnableLambda(critic_node)


In [73]:
from langgraph.graph import StateGraph, END
from langchain_core.runnables import RunnableLambda

graph = StateGraph(ExtractorState)

graph.add_node("actor", actor_runnable)
graph.add_node("critic", critic_runnable)

# Sequence: start at actor → critic
graph.set_entry_point("actor")
graph.add_edge("actor", "critic")

# Conditional: loop back if incorrect
graph.add_conditional_edges(
    "critic",
    lambda state: "actor" if state["feedback"] != "Correct" and state["retries"] < 3 else "__end__"
)

app = graph.compile()

In [74]:
initial_state = {
    "file_name": "CLW_HY25_IP.pdf",
    "retries": 0,
    "bs_result": None,
    "is_result": None,
    "feedback": None,
}

final_state = app.invoke(initial_state)
print(final_state)

Running actor on CLW_HY25_IP.pdf
Processing file: CLW_HY25_IP.pdf
Table 0:
                                                   0  \
0                              Financial performance   
1  Operating EPS of 12.5 cents per security in li...   
2                             $4.62 NTA per security   
3  3.5% like-for-like net property income growth ...   

                                                   1  \
0                              Portfolio performance   
1  Completed asset sales program settled remainin...   
2  99.8% occupancy underpinned by government, ASX...   
3            9.7 year WALE long term income security   

                                                   2  
0                                 Capital management  
1            Completed $50 million security buy-back  
2  31.8% balance sheet gearing within target rang...  
3  Moody's Baa1 stable outlook credit rating reaf...  
Error
Table 1:
                                      0                     1
0          

In [22]:
str(final_state["bs_result"])

"{'total_assets': 5252.5, 'investment_properties': 2773.4, 'total_debt': 1803.2, 'total_liabilities': 1882.7, 'net_assets': 3369.8, 'nta_per_unit': 4.66, 'ticker': 'CLW', 'FP': 'HY25', 'id': 'CLW_HY25'}"